In [1]:
import os
from pathlib import Path
from ultralytics import YOLO
from PIL import Image
import shutil
import pandas as pd
from source import image_id_converter as img_idc
from source import sort_img_files as sif
from source import llm_input as llm_i
from source import llm_output as llm_o
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np

/storage/homefs/sh98e089/venvs/venv-test-clean/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import ollama
import json
import re
import pickle

In [3]:
#root_path = Path('/Users/stephanehess/Documents/CAS_AML/dias_digit_project')
#root_path = Path('/Users/stephanehess/Documents/CAS_AML/dias_digit_project/test_yolo_object_train')

project_path = Path.cwd()
#root_path = (project_path / 'test_LLM_prompt_experiments').resolve()
#root_path = (project_path / '..' / 'test_yolo_object_train').resolve()
#root_path = project_path /'test_llm_img_analysis'
root_path = project_path /'test_LLM_prompt_experiments'
#root_path = project_path
data_path = root_path / 'data'
tif_data_path = root_path / 'data_1'
#data_path = root_path / 'visual_genome_data_all'
jpg_data_path = root_path / 'data_jpg'
#yolo_path = root_path / 'visual_genome_yolo_all'
output_dir_not_photo = root_path / 'not_photo'
output_dir_with_person = root_path / 'with_person'
output_dir_without_person = root_path / 'without_person'

In [4]:
os.listdir(data_path)

['results_prompt_exp_struct_minicpm_2025-10-02.pkl',
 'times_df_prompt_exp_struct_minicpm_2025-10-02 14:00.pkl',
 'results_table_prompt_exp_struct_minicpm_2025-10-02 14:00.pkl',
 'results_prompt_exp_struct_minicpm_2025-10-13.pkl',
 'times_df_prompt_exp_struct_minicpm_2025-10-14 15:59.pkl',
 'labels_mod.csv',
 'labels.csv',
 'results_table_prompt_exp_struct_minicpm_2025-10-14 15:59.pkl',
 '.ipynb_checkpoints',
 'times_df_prompt_exp_struct_minicpm_2025-10-13 17:14.pkl',
 'results_prompt_exp_struct_minicpm_2025-10-25.pkl',
 'ml_metrics_prompt_exp_struct_minicpm_2025-10-04.csv',
 'results_table_prompt_exp_struct_minicpm_2025-10-13 17:14.pkl',
 'times_df_prompt_exp_struct_minicpm_2025-10-25 21:04.pkl',
 'ml_metrics_prompt_exp_struct_minicpm_2025-10-02.csv',
 'results_prompt_exp_struct_minicpm_2025-10-04.pkl',
 'results_table_prompt_exp_struct_minicpm_2025-10-04 15:56.pkl',
 'ml_metrics_prompt_exp_struct_minicpm_2025-10-13.csv',
 'results_prompt_exp_struct_minicpm_2025-10-14.pkl',
 'times_df

### Get list of potentially relevant files: 

In [5]:
csv_files = [f for f in os.listdir(data_path) if f.endswith('.csv')]
pkl_files = [f for f in os.listdir(data_path) if f.endswith('.pkl')]

In [6]:
csv_files

['labels_mod.csv',
 'labels.csv',
 'ml_metrics_prompt_exp_struct_minicpm_2025-10-04.csv',
 'ml_metrics_prompt_exp_struct_minicpm_2025-10-02.csv',
 'ml_metrics_prompt_exp_struct_minicpm_2025-10-13.csv',
 'ml_metrics_prompt_exp_struct_minicpm_2025-10-25.csv',
 'ml_metrics_prompt_exp_struct_minicpm_2025-10-14.csv']

In [7]:
pkl_files

['results_prompt_exp_struct_minicpm_2025-10-02.pkl',
 'times_df_prompt_exp_struct_minicpm_2025-10-02 14:00.pkl',
 'results_table_prompt_exp_struct_minicpm_2025-10-02 14:00.pkl',
 'results_prompt_exp_struct_minicpm_2025-10-13.pkl',
 'times_df_prompt_exp_struct_minicpm_2025-10-14 15:59.pkl',
 'results_table_prompt_exp_struct_minicpm_2025-10-14 15:59.pkl',
 'times_df_prompt_exp_struct_minicpm_2025-10-13 17:14.pkl',
 'results_prompt_exp_struct_minicpm_2025-10-25.pkl',
 'results_table_prompt_exp_struct_minicpm_2025-10-13 17:14.pkl',
 'times_df_prompt_exp_struct_minicpm_2025-10-25 21:04.pkl',
 'results_prompt_exp_struct_minicpm_2025-10-04.pkl',
 'results_table_prompt_exp_struct_minicpm_2025-10-04 15:56.pkl',
 'results_prompt_exp_struct_minicpm_2025-10-14.pkl',
 'times_df_prompt_exp_struct_minicpm_2025-10-04 15:56.pkl',
 'results_table_prompt_exp_struct_minicpm_2025-10-25 21:04.pkl']

### Load labels data: 

In [8]:
label_data_path = os.path.join(data_path, 'labels_mod.csv')
label_data = pd.read_csv(label_data_path)

# Reconvert image ids to integers (e.g. '234') as strings from the form they were saved in (e.g. 'id234' to ensure 
# string data type to deal with duck typing): 
img_ids = list(label_data.image_id)
label_data['image_id'] = img_idc.reconvert_image_ids(img_ids)

label_data.head()

,image_id,with_person,person_recognisable,is_photo,church
0,001,1,1,1,0
1,002,0,0,1,1
2,003,0,0,1,0
3,004,0,0,1,0
4,005,0,0,1,0


### Set timestamp_id:

In [9]:
#timestamp_id = '20251024_114055'
#timestamp_id = '20251025_123936'
timestamp_id = '2025-10-25 21:04'
#timestamp_id = '20251025_152957'

### Set results_tabular (results per image) file name-tag:

In [10]:
#results_name_tag = 'results_tabular_struct_minicpm_'
#results_name_tag = 'results_multi_object_struct_minicpm_'
results_name_tag = 'results_table_prompt_exp_struct_minicpm_'
#results_name_tag = 'results_multi_object_struct_minicpm_'


### Set ml_metrics file name-tag:

In [11]:
#ml_metrics_tag = 'ml_metrics_struct_minicpm_'
#ml_metrics_tag = 'ml_metrics_multi_object_struct_minicpm_'
ml_metrics = 'ml_metrics_prompt_exp_struct_minicpm_2025-10-25.csv'
#ml_metrics_tag = 'ml_metrics_multi_object_struct_minicpm_'

### Set times (duration of tasks) filename tag:

In [12]:
#times_tag = 'times_struct_minicpm_'
#times_tag = 'times_multi_object_struct_minicpm_'
times_tag = 'times_df_prompt_exp_struct_minicpm_'
#times_tag = 'times_multi_object_struct_minicpm_'

### Set response raw data filename tag: 

In [13]:
#responses_tag = 'responses_struct_minicpm_'
#responses_tag = 'responses_multi_object_struct_minicpm_'
responses_tag = 'results_prompt_exp_struct_minicpm_'
#responses_tag = 'responses_multi_object_struct_minicpm_'


### Load ml_metrics data: 

In [15]:
# load ml_metrics:

#filename= ml_metrics_tag + timestamp_id  + '.csv'
filename = ml_metrics
ml_metrics_output_path = os.path.join(data_path, filename)

# Save csv-file: 
#ml_metrics.to_csv(ml_metrics_output_path, index=False)

# Reload saved csv table to check if saving worked:
ml_metrics_reloaded = pd.read_csv(ml_metrics_output_path)
ml_metrics_reloaded.head()

,positives,negatives,true_positives,true_negatives,false_negatives,false_positives,sensitivity,specificity,analysis_name,time_stamp
0,8.0,4.0,8.0,3.0,0.0,1.0,1.0,0.75,is_photo_basic_struct_minicpm,2025-10-25 20:08:21.897406
1,8.0,4.0,8.0,3.0,0.0,1.0,1.0,0.75,is_photo_basic_struct_minicpm,2025-10-25 20:12:02.883091
2,8.0,4.0,8.0,4.0,0.0,0.0,1.0,1.00,is_photo_basic_struct_minicpm,2025-10-25 20:14:49.576028
3,8.0,4.0,8.0,3.0,0.0,1.0,1.0,0.75,is_photo_basic_struct_minicpm,2025-10-25 20:17:37.147219
4,8.0,4.0,8.0,4.0,0.0,0.0,1.0,1.00,is_photo_basic_struct_minicpm,2025-10-25 20:20:23.378384


In [16]:
ml_metrics_reloaded

,positives,negatives,true_positives,true_negatives,false_negatives,false_positives,sensitivity,specificity,analysis_name,time_stamp
0,8.0,4.0,8.0,3.0,0.0,1.0,1.0,0.75,is_photo_basic_struct_minicpm,2025-10-25 20:08:21.897406
1,8.0,4.0,8.0,3.0,0.0,1.0,1.0,0.75,is_photo_basic_struct_minicpm,2025-10-25 20:12:02.883091
2,8.0,4.0,8.0,4.0,0.0,0.0,1.0,1.00,is_photo_basic_struct_minicpm,2025-10-25 20:14:49.576028
3,8.0,4.0,8.0,3.0,0.0,1.0,1.0,0.75,is_photo_basic_struct_minicpm,2025-10-25 20:17:37.147219
4,8.0,4.0,8.0,4.0,0.0,0.0,1.0,1.00,is_photo_basic_struct_minicpm,2025-10-25 20:20:23.378384
5,8.0,4.0,8.0,4.0,0.0,0.0,1.0,1.00,is_photo_intuitive_struct_minicpm,2025-10-25 20:23:09.424511
6,8.0,4.0,8.0,4.0,0.0,0.0,1.0,1.00,is_photo_intuitive_struct_minicpm,2025-10-25 20:25:54.936912
7,8.0,4.0,8.0,4.0,0.0,0.0,1.0,1.00,is_photo_intuitive_struct_minicpm,2025-10-25 20:28:45.539602
8,8.0,4.0,8.0,4.0,0.0,0.0,1.0,1.00,is_photo_intuitive_struct_minicpm,2025-10-25 20:31:34.277568
9,8.0,4.0,8.0,4.0,0.0,0.0,1.0,1.00,is_photo_intuitive_struct_minicpm,2025-10-25 20:34:21.275437


### Load results data: 

In [17]:
# load labels_results:
filename = results_name_tag + timestamp_id + '.pkl'
results_tabular_path = os.path.join(data_path, filename)
# Reload saved dictionary to check if saving worked:
with open(results_tabular_path, 'rb') as f:
   reloaded_results_tabular = pickle.load(f)

In [18]:
print(type(reloaded_results_tabular))
print(len(reloaded_results_tabular))
print(reloaded_results_tabular.keys())


<class 'dict'>
4
dict_keys(['is_photo_basic_struct_minicpm', 'is_photo_intuitive_struct_minicpm', 'is_photo_alternatives_struct_minicpm', 'is_photo_precise_struct_minicpm'])


In [19]:
timestamp_id

'2025-10-25 21:04'

In [20]:
print(type(reloaded_results_tabular[timestamp_id]))
print(len(reloaded_results_tabular[timestamp_id]))
print(reloaded_results_tabular[timestamp_id].keys())

KeyError: '2025-10-25 21:04'

In [21]:
#reloaded_results_tabular['is_photo_basic_struct_minicpm']

In [22]:
type(reloaded_results_tabular[timestamp_id]['predictions'])

KeyError: '2025-10-25 21:04'

In [23]:
reloaded_results_tabular[timestamp_id]['predictions']['image_is_photograph'].head()

KeyError: '2025-10-25 21:04'

In [24]:
label_data_m = label_data.copy()
for key, item in reloaded_results_tabular[timestamp_id]['predictions'].items():
    print(key)
    print(type(item))
    
    label_data_m = label_data_m.merge(item, how='inner', on='image_id')
    print(label_data_m.head())
    

KeyError: '2025-10-25 21:04'

In [25]:
label_data_m.head()

,image_id,with_person,person_recognisable,is_photo,church
0,001,1,1,1,0
1,002,0,0,1,1
2,003,0,0,1,0
3,004,0,0,1,0
4,005,0,0,1,0


In [26]:
print(reloaded_results_tabular.keys())

dict_keys(['is_photo_basic_struct_minicpm', 'is_photo_intuitive_struct_minicpm', 'is_photo_alternatives_struct_minicpm', 'is_photo_precise_struct_minicpm'])


In [28]:
reloaded_results_tabular['is_photo_basic_struct_minicpm'].head()

,image_id,with_person,person_recognisable,is_photo,church,is_photo_pred,time_stamp
0,002,0,0,1,1,1,2025-10-25 20:08:21.897406
1,003,0,0,1,0,1,2025-10-25 20:08:21.897406
2,008,0,0,1,1,1,2025-10-25 20:08:21.897406
3,015,1,1,1,0,1,2025-10-25 20:08:21.897406
4,022,0,0,0,0,0,2025-10-25 20:08:21.897406


### Load raw data (llm responses):

In [29]:

# Define file name: 
filename = responses_tag + timestamp_id + '.pkl'

# Save dictionary with LLM responses:
llm_responses_path = os.path.join(data_path, filename)
# 
# Reload saved dictionary to check if saving worked:
with open(llm_responses_path, 'rb') as f:
   reloaded_image_descr = pickle.load(f)

# Check if original and reloaded dictionary are the same:
#print(len(image_descr))
#print(type(image_descr))
print(type(reloaded_image_descr))
print(len(reloaded_image_descr))

#print(image_descr.keys() == reloaded_image_descr.keys())

FileNotFoundError: [Errno 2] No such file or directory: '/storage/homefs/sh98e089/test_LLM_prompt_experiments/data/results_prompt_exp_struct_minicpm_2025-10-25 21:04.pkl'

In [30]:
reloaded_image_descr.keys()

NameError: name 'reloaded_image_descr' is not defined

In [31]:
reloaded_image_descr['create_prompt_img_type_multi_object_v1'].keys()

NameError: name 'reloaded_image_descr' is not defined

In [32]:
reloaded_image_descr['create_prompt_img_type_multi_object_v1'][timestamp_id].keys()

NameError: name 'reloaded_image_descr' is not defined

In [33]:
reloaded_image_descr['create_prompt_img_type_multi_object_v1'][timestamp_id]['080']

NameError: name 'reloaded_image_descr' is not defined

In [34]:
reloaded_image_descr['create_prompt_img_type_multi_object_v1'][timestamp_id]['101']

NameError: name 'reloaded_image_descr' is not defined

In [35]:
reloaded_image_descr['create_prompt_img_type_multi_object_v1'][timestamp_id]['046']

NameError: name 'reloaded_image_descr' is not defined

In [36]:
dd = {'image_is_photograph': False, 'high_alpine_environment': False, 'person': False, 'glacier': False, 'church': False, 'water_body': True, 'other_objects': ['Map of Belgium and Netherlands', 'Railway lines', 'River (Scheldt, Maas, Rhine)'], 'additional_comments': ''}
print(type(dd))
dd


<class 'dict'>


{'image_is_photograph': False,
 'high_alpine_environment': False,
 'person': False,
 'glacier': False,
 'church': False,
 'water_body': True,
 'other_objects': ['Map of Belgium and Netherlands',
  'Railway lines',
  'River (Scheldt, Maas, Rhine)'],
 'additional_comments': ''}

In [37]:
ml_metrics_reloaded

,positives,negatives,true_positives,true_negatives,false_negatives,false_positives,sensitivity,specificity,analysis_name,time_stamp
0,8.0,4.0,8.0,3.0,0.0,1.0,1.0,0.75,is_photo_basic_struct_minicpm,2025-10-25 20:08:21.897406
1,8.0,4.0,8.0,3.0,0.0,1.0,1.0,0.75,is_photo_basic_struct_minicpm,2025-10-25 20:12:02.883091
2,8.0,4.0,8.0,4.0,0.0,0.0,1.0,1.00,is_photo_basic_struct_minicpm,2025-10-25 20:14:49.576028
3,8.0,4.0,8.0,3.0,0.0,1.0,1.0,0.75,is_photo_basic_struct_minicpm,2025-10-25 20:17:37.147219
4,8.0,4.0,8.0,4.0,0.0,0.0,1.0,1.00,is_photo_basic_struct_minicpm,2025-10-25 20:20:23.378384
5,8.0,4.0,8.0,4.0,0.0,0.0,1.0,1.00,is_photo_intuitive_struct_minicpm,2025-10-25 20:23:09.424511
6,8.0,4.0,8.0,4.0,0.0,0.0,1.0,1.00,is_photo_intuitive_struct_minicpm,2025-10-25 20:25:54.936912
7,8.0,4.0,8.0,4.0,0.0,0.0,1.0,1.00,is_photo_intuitive_struct_minicpm,2025-10-25 20:28:45.539602
8,8.0,4.0,8.0,4.0,0.0,0.0,1.0,1.00,is_photo_intuitive_struct_minicpm,2025-10-25 20:31:34.277568
9,8.0,4.0,8.0,4.0,0.0,0.0,1.0,1.00,is_photo_intuitive_struct_minicpm,2025-10-25 20:34:21.275437


In [38]:
#label_data_1 = reloaded_results_tabular['is_photo_basic_struct_minicpm'].iloc[0:12,:]
#label_data_1

In [39]:
#label_data_3 = reloaded_results_tabular['is_photo_basic_struct_minicpm'].iloc[24:36,:]
#label_data_3

### Load time data:

In [40]:
# load labels_results:
filename = times_tag + timestamp_id + '.pkl'
times_path = os.path.join(data_path, filename)
# Reload saved dictionary to check if saving worked:
with open(times_path, 'rb') as f:
   times_data = pickle.load(f)

In [42]:
pd.DataFrame(times_data)

,analysis_name,time_stamp_start,duration_str,duration_seconds,duration_seconds_str,duration_minutes,duration_minutes_str
0,is_photo_basic_struct_minicpm,2025-10-25 20:08:21.897406,Analysis took: 0 days 00:03:40.815696,220.815696,Analysis took: 220.82 seconds,3.680262,Analysis took: 3.68 minutes
1,is_photo_basic_struct_minicpm,2025-10-25 20:12:02.883091,Analysis took: 0 days 00:02:46.689088,166.689088,Analysis took: 166.69 seconds,2.778151,Analysis took: 2.78 minutes
2,is_photo_basic_struct_minicpm,2025-10-25 20:14:49.576028,Analysis took: 0 days 00:02:47.567520,167.567520,Analysis took: 167.57 seconds,2.792792,Analysis took: 2.79 minutes
3,is_photo_basic_struct_minicpm,2025-10-25 20:17:37.147219,Analysis took: 0 days 00:02:46.226959,166.226959,Analysis took: 166.23 seconds,2.770449,Analysis took: 2.77 minutes
4,is_photo_basic_struct_minicpm,2025-10-25 20:20:23.378384,Analysis took: 0 days 00:02:46.029727,166.029727,Analysis took: 166.03 seconds,2.767162,Analysis took: 2.77 minutes
5,is_photo_intuitive_struct_minicpm,2025-10-25 20:23:09.424511,Analysis took: 0 days 00:02:45.508873,165.508873,Analysis took: 165.51 seconds,2.758481,Analysis took: 2.76 minutes
6,is_photo_intuitive_struct_minicpm,2025-10-25 20:25:54.936912,Analysis took: 0 days 00:02:50.598988,170.598988,Analysis took: 170.60 seconds,2.843316,Analysis took: 2.84 minutes
7,is_photo_intuitive_struct_minicpm,2025-10-25 20:28:45.539602,Analysis took: 0 days 00:02:48.734448,168.734448,Analysis took: 168.73 seconds,2.812241,Analysis took: 2.81 minutes
8,is_photo_intuitive_struct_minicpm,2025-10-25 20:31:34.277568,Analysis took: 0 days 00:02:46.994240,166.994240,Analysis took: 166.99 seconds,2.783237,Analysis took: 2.78 minutes
9,is_photo_intuitive_struct_minicpm,2025-10-25 20:34:21.275437,Analysis took: 0 days 00:02:48.294116,168.294116,Analysis took: 168.29 seconds,2.804902,Analysis took: 2.80 minutes


### Recalculate ml metrics: 

In [ ]:
var_name = 'is_photo'
pred_var_name = 'is_photo_pred'

subsets_and_metrics = llm_o.get_classification_subsets_metrics(label_data_m, var_name, pred_var_name)


In [ ]:
(positives, negatives, true_positives, true_negatives, 
                           false_negatives, false_positives, sensitivity, specificity) = subsets_and_metrics
print('sensitivity:')
print(sensitivity)
print('specificity:')
print(specificity)


### Get number of NaN's:

In [ ]:
num_nans = label_data_m.isna().sum().sum()
num_nans


In [ ]:
num_nulls = label_data_m.isnull().sum().sum()
num_nulls


In [ ]:
label_data_m.isna().sum()

In [ ]:
type(label_data_m.is_photo_pred[0])

### Replace nan's with 0 for confusion matrix (as NaN can be interpreted as 'not recognised as in category x'):

In [ ]:
label_data_m[label_data_m.isna().any(axis=1)]

In [ ]:
label_data_filled = label_data_m.fillna(0)
label_data_filled


In [ ]:
label_data_clean = label_data_m.dropna()
label_data_clean


### Confusion matrix 'is_photo':

#### Recalculate ml metrics: 

In [ ]:
# Recalculate ml metrics: 
var_name = 'is_photo'
pred_var_name = 'is_photo_pred'

subsets_and_metrics = llm_o.get_classification_subsets_metrics(label_data_clean, var_name, pred_var_name)


In [ ]:
(positives, negatives, true_positives, true_negatives, 
                           false_negatives, false_positives, sensitivity, specificity) = subsets_and_metrics
print('sensitivity:')
print(sensitivity)
print('specificity:')
print(specificity)


#### Plot confusion matrix: 

In [ ]:
# Plot confusion matrix: 
cases = true_positives, false_positives, true_negatives, false_negatives, positives, negatives

# llm_o.plot_conf_matrix(labels_results, 'with_person', 'with_person_yolo_pred', cases)
fig = llm_o.save_conf_matrix(label_data_clean, var_name, pred_var_name, cases)


#### Save confusion matrix as pdf:

In [ ]:
# The fig object was created when plotting the confusion matrix
# so now we can use it to save the plot as pdf:
file_name = 'conf_matrix_metrics_' + var_name + '_' + timestamp_id + '.pdf'
conf_matrix_path = data_path / file_name
fig.savefig(conf_matrix_path)
plt.close(fig)


### Confusion matrix 'with_person':

#### Recalculate ml metrics: 

In [ ]:
# Recalculate ml metrics: 
var_name = 'with_person'
pred_var_name = 'with_person_pred'

subsets_and_metrics = llm_o.get_classification_subsets_metrics(label_data_clean, var_name, pred_var_name)


In [ ]:
(positives, negatives, true_positives, true_negatives, 
                           false_negatives, false_positives, sensitivity, specificity) = subsets_and_metrics
print('sensitivity:')
print(sensitivity)
print('specificity:')
print(specificity)


#### Plot confusion matrix:

In [ ]:

cases = true_positives, false_positives, true_negatives, false_negatives, positives, negatives

# llm_o.plot_conf_matrix(labels_results, 'with_person', 'with_person_yolo_pred', cases)
fig = llm_o.save_conf_matrix(label_data_clean, var_name, pred_var_name, cases)

#### Save confusion matrix as pdf:

In [ ]:
# The fig object was created when plotting the confusion matrix
# so now we can use it to save the plot as pdf:
file_name = 'conf_matrix_metrics_' + var_name + '_' + timestamp_id + '.pdf'
conf_matrix_path = data_path / file_name
fig.savefig(conf_matrix_path)
plt.close(fig)

In [ ]:
false_negatives

In [ ]:
false_positives

### Confusion matrix 'with_church':

In [ ]:
# Recalculate ml metrics: 
var_name = 'with_church'
pred_var_name = 'with_church_pred'

subsets_and_metrics = llm_o.get_classification_subsets_metrics(label_data_clean, var_name, pred_var_name)


In [ ]:
(positives, negatives, true_positives, true_negatives, 
                           false_negatives, false_positives, sensitivity, specificity) = subsets_and_metrics
print('sensitivity:')
print(sensitivity)
print('specificity:')
print(specificity)


#### Plot confusion matrix:

In [ ]:

cases = true_positives, false_positives, true_negatives, false_negatives, positives, negatives

# llm_o.plot_conf_matrix(labels_results, 'with_person', 'with_person_yolo_pred', cases)
fig = llm_o.save_conf_matrix(label_data_clean, var_name, pred_var_name, cases)


#### Save confusion matrix as pdf:

In [ ]:
# The fig object was created when plotting the confusion matrix
# so now we can use it to save the plot as pdf:
file_name = 'conf_matrix_metrics_' + var_name + '_' + timestamp_id + '.pdf'
conf_matrix_path = data_path / file_name
fig.savefig(conf_matrix_path)
plt.close(fig)

In [ ]:
false_negatives

In [ ]:
false_positives

### Confusion matrix 'in_high_alpine_environment':

In [ ]:
# Recalculate ml metrics: 
var_name = 'in_high_alpine_environment'
pred_var_name = 'in_high_alpine_environment_pred'

subsets_and_metrics = llm_o.get_classification_subsets_metrics(label_data_clean, var_name, pred_var_name)


In [ ]:
(positives, negatives, true_positives, true_negatives, 
                           false_negatives, false_positives, sensitivity, specificity) = subsets_and_metrics
print('sensitivity:')
print(sensitivity)
print('specificity:')
print(specificity)


#### Plot confusion matrix:

In [ ]:

cases = true_positives, false_positives, true_negatives, false_negatives, positives, negatives

# llm_o.plot_conf_matrix(labels_results, 'with_person', 'with_person_yolo_pred', cases)
fig = llm_o.save_conf_matrix(label_data_clean, var_name, pred_var_name, cases)

#### Save confusion matrix as pdf:

In [ ]:
# The fig object was created when plotting the confusion matrix
# so now we can use it to save the plot as pdf:
file_name = 'conf_matrix_metrics_' + var_name + '_' + timestamp_id + '.pdf'
conf_matrix_path = data_path / file_name
fig.savefig(conf_matrix_path)
plt.close(fig)

In [ ]:
false_negatives

In [ ]:
false_positives

In [ ]:
87/89

In [ ]:
90/99